In [1]:
from astropy.io import fits
import matplotlib.pyplot as plt
import speclite
from speclite import filters
import numpy as np
import pandas as pd
from astropy.io.misc.hdf5 import read_table_hdf5
import h5py
import astropy.units as u
from scipy.integrate import trapezoid
from scipy.interpolate import interp1d

def get_abs_mag(xmmservs_key, xmmservs_cat, xmmservs_seds):
    obsv_wavelength = np.array(xmmservs_seds[str(xmmservs_key)]['wavelength'])*1000. # 1000 nm/micron
    zz = xmmservs_cat.loc[xmmservs_key,'zspec']
    rest_wavelength = obsv_wavelength / (1+zz)
    obsv_specific_lum = np.array(xmmservs_seds[str(xmmservs_key)]['L_lambda_total'])
    rband = filters.load_filter('sdss2010-r')
    rbandwavelength=np.array(rband.wavelength)*0.1 # 0.1 nm per AA
    rbandresponse=np.array(rband.response)
    rbandresponseinterp = interp1d(rbandwavelength,rbandresponse,bounds_error=False,fill_value=0)
    
    Lsun = 3.846e26
    W_per_ergss = 1e-7
    cm_per_Mpc = 3.086e24
    #dist_cm = (3e5*zz)/70. * cm_per_Mpc
    dist_cm = 1e-5 * cm_per_Mpc
    nm_per_AA = 0.1 # nm/AA
    rbandzeropoint_flambda = 278e-11 # ergs/s/cm2/AA
    rbandzeropoint_Llambda = rbandzeropoint_flambda * 4 * np.pi * dist_cm * dist_cm # ergs/s/AA
    rbandzeropoint_Llambda = rbandzeropoint_Llambda * W_per_ergss # W/AA
    rbandzeropoint_Llambda = rbandzeropoint_Llambda / nm_per_AA
    zeropointC = 2.5*np.log10(trapezoid(rbandzeropoint_Llambda*rbandresponseinterp(rest_wavelength),rest_wavelength))
    
    integrand = (obsv_specific_lum)*rbandresponseinterp(rest_wavelength)
    Lr = trapezoid(integrand,rest_wavelength)
    MM = (-2.5*np.log10(Lr)+zeropointC)
    return MM
    

# calculation 

The SEDs provide observed frame wavelength $\lambda_{\rm obs}$ and specific luminosity $L_\lambda(\lambda)$ [W/nm]. To compute the total r-band luminosity of the galaxy, I first shifted the wavelength array to rest frame by computing $\lambda_{\rm rest}=\lambda_{\rm obs}/(1+z)$. I then convolved 

In [4]:
seds = h5py.File('./seds/wcdfs/models_gal_wcdfs.h5', 'r')
catl = pd.read_hdf("XMMSERVS_all.hdf5")

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = './data/seds/wcdfs/models_gal_wcdfs.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [5]:
!pwd

/srv/one/zhutchen/paper3/data


In [ ]:
get_abs_mag(1535, catl, seds)

# Test full sample

In [ ]:
xmmservs = pd.read_hdf("XMMSERVS_all.hdf5")
specz_sample=xmmservs[(xmmservs.zspec>0.002)&(xmmservs.zspec<0.06)].zspec
specz_df = xmmservs[(xmmservs.zspec>0.002)&(xmmservs.zspec<0.06)]

In [ ]:
conv_abs_mags=[]
calc_abs_mags=[]
c_over_H0 = 2.998e+5/70.
for ii in specz_sample.index:
    conv_abs_mags.append(get_abs_mag(ii,catl,seds))
    dist_pc = c_over_H0*xmmservs.loc[ii,:].zspec * 1e6
    calc_abs_mags.append(xmmservs.loc[ii,'mag_R_VOICE'] - 5*np.log10(dist_pc/10))
conv_abs_mags=np.array(conv_abs_mags)
calc_abs_mags=np.array(calc_abs_mags)

In [ ]:
import pickle

In [ ]:
plt.figure()
#tx=np.linspace(0,0.06,10)
tx=np.linspace(-30,0,10)
plt.plot(tx,tx,label='1:1 Line',color='lightgreen')
plt.plot(np.array(conv_abs_mags),calc_abs_mags,'k.')
pickle.dump([np.array(conv_abs_mags),np.array(calc_abs_mags)],open('magmatch.pkl','wb'))

plt.gca().invert_xaxis()
plt.gca().invert_yaxis()
print(np.nanmedian(np.abs(np.array(conv_abs_mags)-np.array(calc_abs_mags))))
plt.xlabel("Rest-frame abs mag. from filter convolution",fontsize=12)
plt.ylabel(r"$m - 5\log_{10}\left(d\, / \, {\rm 10\,pc}\right)$",fontsize=12)
plt.title(r"XMM SERVS $0 < z_{\rm spec} < 0.06$",fontsize=12)
plt.legend(loc='best',fontsize=11)
plt.xlim(-8,-23)
plt.ylim(-8,-23)
plt.show()

In [ ]:
?pickle.dump

In [ ]:
plt.figure()
plt.hist(conv_abs_mags-calc_abs_mags,bins=20)
plt.show()

In [ ]:
eco=pd.read_csv("/srv/two/zhutchen/g3groupfinder/resolve_and_eco/ECOdata_080822.csv")
eco=eco[(eco.logmstar>6)]#&(eco.absrmag<-17.33)]
plt.figure()
plt.scatter(eco.absrmag,eco.logmstar,color='gray',alpha=0.4,s=2, label='ECO')
plt.scatter(calc_abs_mags, np.log10(specz_df['Mstar_gal']),marker='*',color='blue',label=' from m - 5log(d/10)')
plt.scatter(conv_abs_mags, np.log10(specz_df['Mstar_gal']),marker='*',color='red', facecolor='None', label='from SEDs')
#plt.ylim(7,12)
#plt.xlim(-17.33,-23)
plt.gca().invert_xaxis()
plt.xlabel("absolute r-band magnitude")
plt.ylabel("log stellar mass")
plt.legend(loc='best')
plt.show()

In [ ]:
specz_sample

$\lambda$

In [ ]:
xmmservs['Tractor_ID'][0:15]

In [ ]:
seds['1']['L_lambda_total']

In [ ]:
rband.ab_zeropoint

In [ ]:
2.5*np.log10(1e6*3631)

In [ ]:
np.log10(1e6*3631)

In [ ]:
2.5*_

In [ ]:
23.9-_